## SMO Full Platt

In [18]:
import random
from numpy import *

In [38]:
class optStruct:
    def __init__(self, dataMatIn, classLabels, C, toler, kTup):
        self.X = dataMatIn
        self.labelMat = classLabels
        self.C = C
        self.tol = toler
        self.m = shape(dataMatIn)[0]
        self.alphas = mat(zeros((self.m, 1)))
        self.b = 0
        self.eCache = mat(zeros((self.m, 2)))
        self.K = mat(zeros((self.m, self.m)))
        for i in range(self.m):
            self.K[:, i] = kernelTrans(self.X, self.X[i, :], kTup)
    
def loadDataSet(filename):
    dataMat = []
    labelMat = []
    fr = open(filename)
    for line in fr.readlines():
        lineArr = line.strip().split('\t')
        dataMat.append([float(lineArr[0]), float(lineArr[1])])
        labelMat.append(float(lineArr[2]))
    return dataMat, labelMat

def clipAlpha(aj, H, L):
    if aj > H:
        aj = H
    if L > aj:
        aj = L
    return aj

def calcEk(os, k):
    fXk = float(multiply(os.alphas, os.labelMat).T * os.K[:, k] + os.b)
    Ek = fXk - float(os.labelMat[k])
    return Ek

def selectJ(i, os, Ei):
    maxK = -1
    maxDeltaE = 0
    Ej = 0
    os.eCache[i] = [1, Ei]
    validEcacheList = nonzero(os.eCache[:, 0].A)[0]

    if(len(validEcacheList) > 1):
        for k in validEcacheList:
            if k == i:
                continue
            Ek = calcEk(os, k)
            deltaE = abs(Ei - Ek)
            if(deltaE > maxDeltaE):
                maxK = k
                maxDeltaE = deltaE
                Ej = Ek
        return maxK, Ek
    else:
        j = selectJrand(i, os.m)
        Ej = calcEk(os, j)
    return j, Ej

def updateEk(os, k):
    Ek = calcEk(os, k)
    os.eCache[k] = [1, Ek]

def selectJrand(i, m):
    j = i
    while j == i:
        j = int(random.uniform(0, m))
    return j

def innerL(i, os):
    Ei = calcEk(os, i)
    if((os.labelMat[i] * Ei < -os.tol) and (os.alphas[i] < os.C)) or \
        ((os.labelMat[i] * Ei > os.tol) and (os.alphas[i] > 0)):
        j, Ej = selectJ(i, os, Ei)
        alphaIold = os.alphas[i].copy()
        alphaJold = os.alphas[j].copy()
        if(os.labelMat[i] != os.labelMat[j]):
            L = max(0, os.alphas[j] - os.alphas[i])
            H = min(os.C, os.C + os.alphas[j] - os.alphas[i])
        else:
            L = max(0, os.alphas[j] + os.alphas[i] - os.C)
            H = min(os.C, os.alphas[j] + os.alphas[i])

        if L == H:
            print("L == H")
            return 0

        eta = 2.0 * os.K[i, j] - os.K[i, i] - os.K[j, j]
        if eta >= 0:
            print("eta >= 0")
            return 0

        os.alphas[j] -= os.labelMat[j] * (Ei - Ej) / eta
        os.alphas[j] = clipAlpha(os.alphas[j], H, L)

        updateEk(os, j)

        if(abs(os.alphas[j] - alphaJold) < 0.00001):
            print("j not moving enough")
            return 0
        os.alphas[i] += os.labelMat[j] * os.labelMat[i] *\
            (alphaJold - os.alphas[j])
        updateEk(os, i)

        b1 = os.b - Ei - os.labelMat[i] * (os.alphas[i] - alphaIold) * \
            os.K[i, i] - os.labelMat[j] * \
            (os.alphas[j] - alphaJold) * os.K[i, j]

        b2 = os.b - Ej - os.labelMat[i] * (os.alphas[i] - alphaIold) * \
            os.K[i, j] - os.labelMat[j] * \
            (os.alphas[j] - alphaJold) * os.K[j, j]

        if(0 < os.alphas[i]) and (os.C > os.alphas[i]):
            os.b = b1
        elif (0 < os.alphas[j]) and (os.C > os.alphas[j]):
            os.b = b2
        else:
            os.b = (b1 + b2) / 2.0
        return 1
    else:
        return 0

def smoP(dataMatIn, classLabels, C, toler, maxIter, kTup=('lin', 0)):
    os = optStruct(mat(dataMatIn), mat(classLabels).transpose(), C, toler, kTup)
    itr = 0
    entireSet = True
    alphaPairsChanged = 0
    
    while(itr < maxIter) and ((alphaPairsChanged > 0) or (entireSet)):
        alphaPairsChanged = 0
        if entireSet:
            for i in range(os.m):
                alphaPairsChanged += innerL(i, os)
            print("Fullset, iter: %d i: %d, pairs changed: %d" % (itr, i, alphaPairsChanged))
            itr += 1
        else:
            nonBoundIs = nonzero((os.alphas.A > 0) * (os.alphas.A < C))[0]
            for i in nonBoundIs:
                alphaPairsChanged += innerL(i, os)
                print("Nonbound, iter: %d i: %d, pairs changed: %d" % (itr, i, alphaPairsChanged))
            itr += 1
        
        if entireSet:
            entireSet = False
        elif (alphaPairsChanged == 0):
            entireSet = True
        print("Iteration number: %d" % itr)
    
    return os.b, os.alphas

In [34]:
dataArr, labelArr = loadDataSet('testSet.txt')

In [35]:
b, alphas = smoP(dataArr, labelArr, 0.6, 0.001, 40)

j not moving enough
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
L == H
j not moving enough
Fullset, iter: 0 i: 99, pairs changed: 12
Iteration number: 1
Nonbound, iter: 1 i: 0, pairs changed: 1
Nonbound, iter: 1 i: 4, pairs changed: 2
Nonbound, iter: 1 i: 5, pairs changed: 3
Nonbound, iter: 1 i: 6, pairs changed: 4
Nonbound, iter: 1 i: 8, pairs changed: 5
Nonbound, iter: 1 i: 17, pairs changed: 6
Nonbound, iter: 1 i: 18, pairs changed: 7
Nonbound, iter: 1 i: 25, pairs changed: 8
Nonbound, iter: 1 i: 26, pairs changed: 9
L == H
Nonbound, iter: 1 i: 55, pairs changed: 9
j not moving enough
Nonbound, iter: 1 i: 94, pairs changed: 9
Nonbound, iter: 1 i: 97, pairs changed: 10
Iteration number: 2
Nonbound, iter: 2 i: 0, pairs changed: 1
Nonbound, iter: 2 i: 2, pairs changed: 1
Nonbound, iter: 2 i: 4, pairs changed: 2
Nonbound, iter: 2 i: 5, pairs changed: 3
Nonbound, iter: 2 i: 8, pairs changed: 4
Nonbound, iter: 2 i: 17, pairs changed: 5
Nonbound, iter: 2 i: 25, pa

Nonbound, iter: 18 i: 97, pairs changed: 5
Iteration number: 19
Nonbound, iter: 19 i: 0, pairs changed: 1
j not moving enough
Nonbound, iter: 19 i: 2, pairs changed: 1
Nonbound, iter: 19 i: 17, pairs changed: 2
j not moving enough
Nonbound, iter: 19 i: 25, pairs changed: 2
j not moving enough
Nonbound, iter: 19 i: 26, pairs changed: 2
Nonbound, iter: 19 i: 52, pairs changed: 3
j not moving enough
Nonbound, iter: 19 i: 92, pairs changed: 3
Nonbound, iter: 19 i: 94, pairs changed: 4
Nonbound, iter: 19 i: 97, pairs changed: 5
Iteration number: 20
Nonbound, iter: 20 i: 0, pairs changed: 1
j not moving enough
Nonbound, iter: 20 i: 2, pairs changed: 1
Nonbound, iter: 20 i: 17, pairs changed: 2
j not moving enough
Nonbound, iter: 20 i: 25, pairs changed: 2
j not moving enough
Nonbound, iter: 20 i: 26, pairs changed: 2
j not moving enough
Nonbound, iter: 20 i: 52, pairs changed: 2
L == H
Nonbound, iter: 20 i: 92, pairs changed: 2
L == H
Nonbound, iter: 20 i: 94, pairs changed: 2
j not moving e

In [36]:
def calcWs(alphas, dataArr, classLabels):
    X = mat(dataArr)
    labelMat = mat(classLabels).transpose()
    m, n = shape(X)
    w = zeros((n, 1))
    for i in range(m):
        w += multiply(alphas[i] * labelMat[i], X[i, :].T)
    return w

In [37]:
def kernelTrans(X, A, kTup):
    m, n = shape(X)
    K = mat(zeros((m, 1)))
    
    if kTup[0] == 'lin':
        K = X * A.T
    elif kTup[0] == 'rbf':
        for j in range(m):
            deltaRow = X[j, :] - A
            K[j] = deltaRow * deltaRow.T
        K = exp(K / (-1 * kTup[1] ** 2))
    else:
        raise NameError('Kernel not recognized')
    return K

In [39]:
def testRBF(k1 = 1.3):
    dataArr, labelArr = loadDataSet('testSetRBF.txt')
    b, alphas = smoP(dataArr, labelArr, 200, 0.0001, 10000, ('rbf', k1))
    datMat = mat(dataArr)
    labelMat = mat(labelArr).transpose()
    svInd = nonzero(alphas.A > 0)[0]
    sVs = datMat[svInd]
    labelSV = labelMat[svInd]
    print('There are %d support vectors' % shape(sVs)[0])
    m, n = shape(datMat)
    errorCount = 0
    for i in range(m):
        kernelEval = kernelTrans(sVs, datMat[i, :], ("rbf", k1))
        predict = kernelEval.T * multiply(labelSV, alphas[svInd]) + b
        if sign(predict) != sign(labelArr[i]):
            errorCount += 1
        
    print("Training error: %f" %(float(errorCount) / m))

In [40]:
testRBF()

L == H
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
j not moving enough
L == H
Fullset, iter: 0 i: 99, pairs changed: 70
Iteration number: 1
Nonbound, iter: 1 i: 0, pairs changed: 1
Nonbound, iter: 1 i: 1, pairs changed: 2
Nonbound, iter: 1 i: 3, pairs changed: 3
Nonbound, iter: 1 i: 4, pairs changed: 4
Nonbound, iter: 1 i: 5, pairs changed: 5
Nonbound, iter: 1 i: 8, pairs changed: 6
Nonbound, iter: 1 i: 10, pairs changed: 7
Nonbound, iter: 1 i: 11, pairs changed: 8
Nonbound, iter: 1 i: 13, pairs changed: 9
Nonbound, iter: 1 i: 14, pairs changed: 10
Nonbound, iter: 1 i: 15, pairs changed: 11
Nonbound, iter: 1 i: 16, pairs changed: 12
Nonbound, iter: 1 i: 17, pairs changed: 13
Nonbound, iter: 1 i: 18, pairs changed: 14
Nonbound, iter: 1 i: 21, pairs changed: 15
Nonbound, iter: 1 i: 22, pairs changed: 16
Nonbound, iter: 1 i: 24, pairs changed: 17
Nonbound, iter: 1 i: 28, pairs changed: 18
Nonbo

Nonbound, iter: 3 i: 73, pairs changed: 50
Nonbound, iter: 3 i: 74, pairs changed: 51
Nonbound, iter: 3 i: 75, pairs changed: 52
Nonbound, iter: 3 i: 76, pairs changed: 53
Nonbound, iter: 3 i: 77, pairs changed: 54
Nonbound, iter: 3 i: 79, pairs changed: 55
Nonbound, iter: 3 i: 80, pairs changed: 56
Nonbound, iter: 3 i: 81, pairs changed: 57
Nonbound, iter: 3 i: 83, pairs changed: 58
Nonbound, iter: 3 i: 86, pairs changed: 59
Nonbound, iter: 3 i: 87, pairs changed: 60
Nonbound, iter: 3 i: 89, pairs changed: 61
Nonbound, iter: 3 i: 93, pairs changed: 62
Nonbound, iter: 3 i: 94, pairs changed: 63
Nonbound, iter: 3 i: 95, pairs changed: 64
Nonbound, iter: 3 i: 96, pairs changed: 65
Nonbound, iter: 3 i: 98, pairs changed: 66
Nonbound, iter: 3 i: 99, pairs changed: 67
Iteration number: 4
Nonbound, iter: 4 i: 0, pairs changed: 1
Nonbound, iter: 4 i: 1, pairs changed: 2
Nonbound, iter: 4 i: 3, pairs changed: 3
Nonbound, iter: 4 i: 5, pairs changed: 4
Nonbound, iter: 4 i: 8, pairs changed: 5
N

Nonbound, iter: 6 i: 63, pairs changed: 41
Nonbound, iter: 6 i: 65, pairs changed: 42
Nonbound, iter: 6 i: 68, pairs changed: 43
Nonbound, iter: 6 i: 69, pairs changed: 44
Nonbound, iter: 6 i: 70, pairs changed: 45
Nonbound, iter: 6 i: 71, pairs changed: 46
Nonbound, iter: 6 i: 72, pairs changed: 47
Nonbound, iter: 6 i: 73, pairs changed: 48
Nonbound, iter: 6 i: 74, pairs changed: 49
Nonbound, iter: 6 i: 75, pairs changed: 50
Nonbound, iter: 6 i: 76, pairs changed: 51
Nonbound, iter: 6 i: 77, pairs changed: 52
Nonbound, iter: 6 i: 79, pairs changed: 53
Nonbound, iter: 6 i: 80, pairs changed: 54
Nonbound, iter: 6 i: 81, pairs changed: 55
Nonbound, iter: 6 i: 83, pairs changed: 56
Nonbound, iter: 6 i: 86, pairs changed: 57
Nonbound, iter: 6 i: 87, pairs changed: 58
Nonbound, iter: 6 i: 89, pairs changed: 59
Nonbound, iter: 6 i: 93, pairs changed: 60
Nonbound, iter: 6 i: 94, pairs changed: 61
Nonbound, iter: 6 i: 95, pairs changed: 62
Nonbound, iter: 6 i: 96, pairs changed: 63
Nonbound, i

Nonbound, iter: 9 i: 68, pairs changed: 41
Nonbound, iter: 9 i: 69, pairs changed: 42
Nonbound, iter: 9 i: 70, pairs changed: 43
Nonbound, iter: 9 i: 71, pairs changed: 44
Nonbound, iter: 9 i: 72, pairs changed: 45
Nonbound, iter: 9 i: 73, pairs changed: 46
Nonbound, iter: 9 i: 74, pairs changed: 47
Nonbound, iter: 9 i: 75, pairs changed: 48
Nonbound, iter: 9 i: 76, pairs changed: 49
Nonbound, iter: 9 i: 77, pairs changed: 50
Nonbound, iter: 9 i: 79, pairs changed: 51
Nonbound, iter: 9 i: 80, pairs changed: 52
Nonbound, iter: 9 i: 81, pairs changed: 53
j not moving enough
Nonbound, iter: 9 i: 83, pairs changed: 53
j not moving enough
Nonbound, iter: 9 i: 86, pairs changed: 53
Nonbound, iter: 9 i: 87, pairs changed: 54
Nonbound, iter: 9 i: 89, pairs changed: 55
Nonbound, iter: 9 i: 93, pairs changed: 56
Nonbound, iter: 9 i: 94, pairs changed: 57
Nonbound, iter: 9 i: 95, pairs changed: 58
Nonbound, iter: 9 i: 96, pairs changed: 59
Nonbound, iter: 9 i: 98, pairs changed: 60
Nonbound, iter

j not moving enough
Nonbound, iter: 12 i: 39, pairs changed: 6
j not moving enough
Nonbound, iter: 12 i: 40, pairs changed: 6
j not moving enough
Nonbound, iter: 12 i: 41, pairs changed: 6
j not moving enough
Nonbound, iter: 12 i: 42, pairs changed: 6
j not moving enough
Nonbound, iter: 12 i: 43, pairs changed: 6
j not moving enough
Nonbound, iter: 12 i: 44, pairs changed: 6
j not moving enough
Nonbound, iter: 12 i: 45, pairs changed: 6
j not moving enough
Nonbound, iter: 12 i: 47, pairs changed: 6
Nonbound, iter: 12 i: 48, pairs changed: 7
j not moving enough
Nonbound, iter: 12 i: 49, pairs changed: 7
j not moving enough
Nonbound, iter: 12 i: 50, pairs changed: 7
j not moving enough
Nonbound, iter: 12 i: 53, pairs changed: 7
j not moving enough
Nonbound, iter: 12 i: 54, pairs changed: 7
j not moving enough
Nonbound, iter: 12 i: 56, pairs changed: 7
Nonbound, iter: 12 i: 57, pairs changed: 8
j not moving enough
Nonbound, iter: 12 i: 58, pairs changed: 8
Nonbound, iter: 12 i: 59, pairs 

...

j not moving enough
Nonbound, iter: 205 i: 48, pairs changed: 0
j not moving enough
Nonbound, iter: 205 i: 49, pairs changed: 0
j not moving enough
Nonbound, iter: 205 i: 50, pairs changed: 0
Nonbound, iter: 205 i: 53, pairs changed: 0
j not moving enough
Nonbound, iter: 205 i: 56, pairs changed: 0
j not moving enough
Nonbound, iter: 205 i: 57, pairs changed: 0
Nonbound, iter: 205 i: 58, pairs changed: 0
Nonbound, iter: 205 i: 59, pairs changed: 1
j not moving enough
Nonbound, iter: 205 i: 60, pairs changed: 1
j not moving enough
Nonbound, iter: 205 i: 61, pairs changed: 1
j not moving enough
Nonbound, iter: 205 i: 62, pairs changed: 1
Nonbound, iter: 205 i: 63, pairs changed: 2
j not moving enough
Nonbound, iter: 205 i: 65, pairs changed: 2
j not moving enough
Nonbound, iter: 205 i: 68, pairs changed: 2
j not moving enough
Nonbound, iter: 205 i: 69, pairs changed: 2
j not moving enough
Nonbound, iter: 205 i: 70, pairs changed: 2
j not moving enough
Nonbound, iter: 205 i: 71, pairs cha

j not moving enough
Nonbound, iter: 208 i: 21, pairs changed: 0
j not moving enough
Nonbound, iter: 208 i: 22, pairs changed: 0
Nonbound, iter: 208 i: 24, pairs changed: 0
j not moving enough
Nonbound, iter: 208 i: 28, pairs changed: 0
j not moving enough
Nonbound, iter: 208 i: 30, pairs changed: 0
Nonbound, iter: 208 i: 31, pairs changed: 0
j not moving enough
Nonbound, iter: 208 i: 33, pairs changed: 0
Nonbound, iter: 208 i: 36, pairs changed: 0
j not moving enough
Nonbound, iter: 208 i: 39, pairs changed: 0
j not moving enough
Nonbound, iter: 208 i: 40, pairs changed: 0
j not moving enough
Nonbound, iter: 208 i: 44, pairs changed: 0
j not moving enough
Nonbound, iter: 208 i: 47, pairs changed: 0
j not moving enough
Nonbound, iter: 208 i: 48, pairs changed: 0
j not moving enough
Nonbound, iter: 208 i: 49, pairs changed: 0
j not moving enough
Nonbound, iter: 208 i: 50, pairs changed: 0
Nonbound, iter: 208 i: 53, pairs changed: 0
j not moving enough
Nonbound, iter: 208 i: 56, pairs cha

j not moving enough
Nonbound, iter: 211 i: 10, pairs changed: 0
j not moving enough
Nonbound, iter: 211 i: 13, pairs changed: 0
j not moving enough
Nonbound, iter: 211 i: 14, pairs changed: 0
Nonbound, iter: 211 i: 17, pairs changed: 0
j not moving enough
Nonbound, iter: 211 i: 20, pairs changed: 0
j not moving enough
Nonbound, iter: 211 i: 21, pairs changed: 0
j not moving enough
Nonbound, iter: 211 i: 22, pairs changed: 0
Nonbound, iter: 211 i: 24, pairs changed: 0
j not moving enough
Nonbound, iter: 211 i: 28, pairs changed: 0
j not moving enough
Nonbound, iter: 211 i: 30, pairs changed: 0
Nonbound, iter: 211 i: 31, pairs changed: 0
j not moving enough
Nonbound, iter: 211 i: 33, pairs changed: 0
Nonbound, iter: 211 i: 36, pairs changed: 0
j not moving enough
Nonbound, iter: 211 i: 39, pairs changed: 0
j not moving enough
Nonbound, iter: 211 i: 40, pairs changed: 0
j not moving enough
Nonbound, iter: 211 i: 44, pairs changed: 0
j not moving enough
Nonbound, iter: 211 i: 47, pairs cha

Nonbound, iter: 214 i: 63, pairs changed: 2
Nonbound, iter: 214 i: 65, pairs changed: 2
j not moving enough
Nonbound, iter: 214 i: 68, pairs changed: 2
Nonbound, iter: 214 i: 69, pairs changed: 2
Nonbound, iter: 214 i: 70, pairs changed: 2
j not moving enough
Nonbound, iter: 214 i: 71, pairs changed: 2
Nonbound, iter: 214 i: 72, pairs changed: 3
Nonbound, iter: 214 i: 73, pairs changed: 4
j not moving enough
Nonbound, iter: 214 i: 74, pairs changed: 4
j not moving enough
Nonbound, iter: 214 i: 75, pairs changed: 4
Nonbound, iter: 214 i: 76, pairs changed: 4
j not moving enough
Nonbound, iter: 214 i: 77, pairs changed: 4
Nonbound, iter: 214 i: 79, pairs changed: 4
j not moving enough
Nonbound, iter: 214 i: 80, pairs changed: 4
Nonbound, iter: 214 i: 81, pairs changed: 4
j not moving enough
Nonbound, iter: 214 i: 83, pairs changed: 4
j not moving enough
Nonbound, iter: 214 i: 86, pairs changed: 4
Nonbound, iter: 214 i: 87, pairs changed: 4
j not moving enough
Nonbound, iter: 214 i: 94, p

j not moving enough
Nonbound, iter: 219 i: 39, pairs changed: 0
j not moving enough
Nonbound, iter: 219 i: 40, pairs changed: 0
j not moving enough
Nonbound, iter: 219 i: 44, pairs changed: 0
j not moving enough
Nonbound, iter: 219 i: 47, pairs changed: 0
Nonbound, iter: 219 i: 48, pairs changed: 0
j not moving enough
Nonbound, iter: 219 i: 49, pairs changed: 0
j not moving enough
Nonbound, iter: 219 i: 50, pairs changed: 0
Nonbound, iter: 219 i: 53, pairs changed: 0
j not moving enough
Nonbound, iter: 219 i: 56, pairs changed: 0
Nonbound, iter: 219 i: 57, pairs changed: 0
Nonbound, iter: 219 i: 58, pairs changed: 0
Nonbound, iter: 219 i: 59, pairs changed: 0
j not moving enough
Nonbound, iter: 219 i: 60, pairs changed: 0
j not moving enough
Nonbound, iter: 219 i: 61, pairs changed: 0
j not moving enough
Nonbound, iter: 219 i: 62, pairs changed: 0
Nonbound, iter: 219 i: 63, pairs changed: 0
Nonbound, iter: 219 i: 65, pairs changed: 0
j not moving enough
Nonbound, iter: 219 i: 68, pairs